а) Осуществить моделирование n = 1000 реализаций БСВ с помощью мультипликативного
конгруэнтного метода (МКМ) с параметрами a 0 = a 01 , β = max{c 1 , M – c 1 },
M = 2 31 и вывести 100-ый, 900-ый и 1000-ый элементы сгенерированной последовательности.

б) Осуществить моделирование n = 1000 реализаций БСВ с помощью метода
Макларена-Марсальи, используя в качестве простейших датчиков БСВ датчики D 1 –
датчик из первого задания, D 2 – датчик по методу МКМ с параметрами a 0 = a 02 ,
β = max{c 2 , M – c 2 }, M = 2 31 , K – объем вспомогательной таблицы и вывести
100-ый, 900-ый и 1000-ый элементы сгенерированной последовательности.

в) Проверить точность моделировнаия обоиз датчиков с помощью критерия согласия Колмогорова и чи-квадрат критерия Пирсона с уровнем значимости e=0.05

In [14]:
from typing import List
from bisect import bisect_right
import numpy as np

In [15]:
# 16 варыянт
M = 2 ** 31
n = 10 ** 3
a01 = 80202397
c1 = 264561431
b1 = max(c1, M - c1)
a02 = 289809159
c2 = 90281719
b2 = max(c2, M - c2)
K = 128

In [16]:
def mpcm(n: int, a0: int, b: int, M: int):
    a = a0
    for i in range(n):
        a = (b * a) % M
        yield a / M

def mcl_mrsl(n: int, b: List[int], c: List[int], k: int):
    v = b[:k]
    for i in range(n):
        s = int(c[i] * k)
        yield v[s]
        v[s] = b[i + k]

In [17]:
test01 = list(mpcm(n + K, a01, b1, M))
test02 = list(mpcm(n + K, a02, b2, M))
test2 = list(mcl_mrsl(n, test01, test02, K))
print(f'mpcm:\n1st: {test01[0]}, 15th: {test01[14]}, \nmean:{np.mean(test01)}, std:{np.std(test01)}')
print(f'mcl_mrsl:\n1st: {test2[0]}, 15th: {test2[14]}, \nmean:{np.mean(test2)}, std:{np.std(test2)}')

mpcm:
1st: 0.1733270757831633, 15th: 0.6222331621684134, 
mean:0.4919283538429291, std:0.2918025937829086
mcl_mrsl:
1st: 0.7692250036634505, 15th: 0.8472271435894072, 
mean:0.49168137151002883, std:0.29206808132248546


In [18]:
def sort_and_count(n, rand_seq):
    seq = sorted(rand_seq, key=float)
    size = len(seq)
    segment_size = int(size / n)
    probability = 1 / n
    frequencies = []
    j = 0
    for i in range(n):
        count = 0    
        while j < size and seq[j] < (i + 1) / n:
            j += 1
            count += 1
        frequencies.append(count)
    return frequencies, segment_size, probability

def chi_square(frequencies, size, probability, delta):
    chi = 0
    for frequency in frequencies:
        chi += (frequency - probability * size) ** 2 / (probability * size)
    print('chi-square : {}, delta : {}'.format(chi, delta))
    
    return chi < delta

In [19]:
delta_10_005 = 16.919
delta_40_005 = 54.572
k = 10
frequencies, segment_size, probability = sort_and_count(k, test01)
print('frequencies : {}'.format(frequencies))
print('segment size : {}'.format(segment_size))
print('probability : {}'.format(probability))
chi_square(frequencies, len(test01), probability, delta_10_005)

frequencies : [133, 104, 121, 108, 104, 114, 116, 99, 111, 118]
segment size : 112
probability : 0.1
chi-square : 7.851063829787233, delta : 16.919


True

In [20]:
frequencies, segment_size, probability = sort_and_count(k, test2)
print('frequencies : {}'.format(frequencies))
print('segment size : {}'.format(segment_size))
print('probability : {}'.format(probability))
chi_square(frequencies, len(test2), probability, delta_10_005)

frequencies : [122, 89, 106, 93, 93, 103, 105, 90, 92, 107]
segment size : 100
probability : 0.1
chi-square : 9.860000000000001, delta : 16.919


True

In [26]:
def kalmahorau(data):
    delta = 1.358
    n = len(data)
    empirical = np.sort(data)
    theoretical = np.arange(n) / n
    diff = np.max(np.abs(empirical - theoretical))
    value = diff * np.sqrt(n)
    print('kalmahorau : {}, delta : {}'.format(value, delta))
    return value < delta

In [27]:
kalmahorau(test01)

kalmahorau : 0.8733757600668444, delta : 1.358


True

In [28]:
kalmahorau(test2)

kalmahorau : 0.7756433530708249, delta : 1.358


True